In [318]:
import pandas

In [319]:
df = pandas.read_csv('frappe.csv', sep="\t")
meta_app = pandas.read_csv('meta.csv', sep="\t")
df = df.merge(meta_app, on='item')

In [320]:
import numpy as np
import scipy.stats
import scipy.spatial
import math
from math import sqrt

In [322]:
ui = np.zeros((957,4082))

In [323]:
for i in range(957):
    ii = list(set(df.loc[df['user'] == i]['item']))
    for j in ii:
        dfi = df.loc[df['user'] == i]
        dfij = dfi.loc[dfi['item'] == j]
        rij = sum(dfij['cnt'])
        ui[i,j] = rij

In [324]:
print ui

[[  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   4.42200000e+03   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.40000000e+01 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]


In [ ]:
def simps(userd):
    '''
    It will accept the userd as arguments.
    and return the ss as a list.
    Input: userd (user's score for each app, array).
    Output: ss (list of similarity scores for each user with the given user)
    '''
    ss = []
    for i in range(957):
        index = list(set(np.nonzero(ui[i,])[0])&set(np.nonzero(userd)[0])) #the app index for both rated
        uset = ui[i,][index]
        usen = userd[index]
        if not uset.size:
            rr = 0
        else:
            #use cosine distance to get the similarity score
            xx = uset.dot(usen.T)
            x1 = sqrt(sum([x**2 for x in uset]))
            x2 = sqrt(sum([x**2 for x in usen]))
            rr = xx/(x1*x2)
        ss.append(rr)
    return ss    

In [349]:
def rec(userd):
    #unit the similarity score
    usc = simps(userd)/sum(simps(userd))
    isc = np.zeros((957,4082))
    for i in range(957):
        isc[i,] = ui[i,]*usc[i]
    #get the recommend score for each app
    iscf = np.sum(isc,axis = 0)
    #remove the apps' score which the user has used
    index = list(np.nonzero(userd)[0])
    for j in index:
        iscf[j] = 0
    #recommend the highest score app to user
    iid = scipy.argmax(iscf)
    nre = set(df.loc[df['item'] == iid]['name'])
    return nre    

In [352]:
rec(ui[0,])

{'WhatsApp Messenger'}